In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
#pip install napari[all] napari_video
import napari
from napari_video.napari_video import VideoReaderNP

In [10]:
vdos = []

In [11]:
vdos

[]

In [6]:
!git clone https://github.com/Hzzone/pytorch-openpose
%cd pytorch-openpose
import matplotlib.pyplot as plt
import copy
import numpy as np
from src import model
from src import util
from src.body import Body
from src.hand import Hand

!cp /kaggle/input/openpose-pretrained-models/body_pose_model.pth model
!cp /kaggle/input/openpose-pretrained-models/hand_pose_model.pth model

body_estimation = Body('model/body_pose_model.pth')
hand_estimation = Hand('model/hand_pose_model.pth')

Cloning into 'pytorch-openpose'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 154 (delta 69), reused 152 (delta 67), pack-reused 0
Receiving objects: 100% (154/154), 20.18 MiB | 31.12 MiB/s, done.
Resolving deltas: 100% (69/69), done.
/kaggle/working/pytorch-openpose


In [7]:
vdos.append(VideoReaderNP('/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_2987.MOV'))


NameError: name 'VideoReaderNP' is not defined

In [ ]:
def get_all_hand_peaks(hands_list,oriImg):
    all_hand_peaks = []
    for x, y, w, is_left in hands_list:
        peaks = hand_estimation(oriImg[y:y+w, x:x+w, :]) #pretrained model, last layer: convolution_param-> num_output: 22, 21 Keypoints
        #peaks is coordinates wrt x,y top left.
        #adjust coordinates of keypoints
        peaks[:, 0] = np.where(peaks[:, 0]==0, peaks[:, 0], peaks[:, 0]+x)
        peaks[:, 1] = np.where(peaks[:, 1]==0, peaks[:, 1], peaks[:, 1]+y) 
        all_hand_peaks.append(peaks)
    return all_hand_peaks


In [ ]:
#figure ret val and vdos_keypoints, vdo_keypoints
def generate_hand_keypoints_for_one_video(video):
    vdo_canvas = []
    vdo_keypoints = []
    Num_of_frames = len(video)
    print(i,Num_of_frames)
    for f in range(Num_of_frames):
        oriImg = video[f]

        candidate, subset = body_estimation(oriImg)
        canvas = copy.deepcopy(oriImg)
        hands_list = util.handDetect(candidate, subset, oriImg)
        all_hand_peaks = get_all_hand_peaks(hands_list,oriImg)
        canvas = util.draw_handpose(canvas, all_hand_peaks)
        vdo_canvas.append(canvas)
        #vdos_keypoints.append(np.array(vdo_keypoints,dtype = 'float32'))
    return vdo_canvas

In [ ]:
def generate_hand_keypoints_canvas_for_one_video(video):
    vdo_canvas = []
    Num_of_frames = len(video)
    for f in range(Num_of_frames):
        oriImg = video[f]

        candidate, subset = body_estimation(oriImg)
        canvas = copy.deepcopy(oriImg)
        hands_list = util.handDetect(candidate, subset, oriImg)
        all_hand_peaks = get_all_hand_peaks(hands_list,oriImg)
        canvas = util.draw_handpose(canvas, all_hand_peaks)
        vdo_canvas.append(canvas)
    return vdo_canvas

This function should take one vdo and return its hand keypoint, then append those keypoints to X. check Vee's paf codes too.

When it fails to detect hands, co ordinates are 0,0, we want that to be ignored. we dont want the model to be trained on that.

In [13]:
vdo_canvas = generate_hand_keypoints_for_one_video(vdos[0])

NameError: name 'i' is not defined

In [ ]:
len(vdo_canvas)
#76X288X512X3
#frames x 

In [ ]:
labels = ['Bird','Black','Car','Dog','Fall','Father','Good Morning', 'Red','Summer','White','loud','quiet','happy','long','short','big large', 'small little','hot', 'new','dry','good','Cow','Hat', 'T-Shirt', 'Shoes', 'Monday', 'Year', 'Time', 'Fan', 'Cell phone', 'Hello', 'Thank you', 'Window', 'Pen', 'Paint', 'Teacher', 'Priest', 'train ticket', 'Brother', 'Boy', ' Girl', ' Bank', 'I', 'it','you', 'Election', 'Death', 'Court', 'House', 'Store or Shop']

In [ ]:
lines = [24,15,16,13,12,18,17,18,11,19,15,18,18,15,16,17,18,15,18,21,18,18,14,17,16,12,8,12,12,10,14,14,11,12,12,11,14,14,16,17,18,16,18,12,18,12,9,20,20,18]

In [ ]:
len(labels)

In [24]:
vdos

[/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_2987.MOV with 76 frames of size (1080, 1920, 3) at 25.00 fps]

In [ ]:
y = []

In [ ]:
for i in range(49):
    for j in range(1,lines[i]):
        y.append(i)

In [ ]:
len(y)

Dask Trials
https://napari.org/stable/tutorials/processing/dask.html
Tried:
1. glob returns an array of all filenames, imread takes glob like string and not the array.
iglob does return an iterator but it still isn't working. It says:
Could not detect your file type because it did not have an extension. Try specifying a loader class, e.g. Video(<generator object _iglob at 0x7f5ec4c5c1d0>)
To Do: Figure out how to specify a loader class.


2. Dask requirements.txt or environment.yml not yet run
I tried to use the binder environment (requires ur repo to be public, did that) butt it says:
package libgcc-ng-12.2.0-h65d4601_19 requires _libgcc_mutex 0.1 conda_forge, but none of the providers can be installed

To Do: Install all the requirements and run

3. Read vdos using napari and use dask array.. how to increase dask array...
TO DO: checkout AI4bharat's work on include.

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
#In console run following:
#pip install scikit-image
#pip install graphviz
#pip install dask-image
#pip install imageio[pyav]
import dask
import matplotlib
import numpy
#import dask_image.imread
#import dask_image.ndfilters
#import dask_image.ndmeasure
import dask.array as da
import glob

In [23]:
filename_pattern

['/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_2987.MOV',
 '/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_2988.MOV',
 '/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_2989.MOV',
 '/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3011.MOV',
 '/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3012.MOV',
 '/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3013.MOV',
 '/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3037.MOV',
 '/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3038.MOV',
 '/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3040.MOV',
 '/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3070.MOV',
 '/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3071.MOV',
 '/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3072.MOV',
 '/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3094.MOV',
 '/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3095.MOV',
 '/kaggle/input/include/Animals_1of2/Animals/4. 

In [4]:
filename_pattern1

'/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI*.MOV'

In [3]:
filename_pattern1 = os.path.join('/kaggle/input/include/Animals_1of2/Animals/4. Bird','MVI*.MOV')

In [42]:
tiled_astronaut_images = dask_image.imread.imread(filename_pattern1)

ValueError: Could not find a backend to open `/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_2987.MOV`` with iomode `ri`.

In [24]:
print('\nNamed with wildcard *:')
for name in glob.iglob('/kaggle/input/include/Animals_1of2/Animals/4. Bird/*.M*'):
    print(name)


Named with wildcard *:
/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3040.MOV
/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3070.MOV
/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3011.MOV
/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_2989.MOV
/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_8568.MP4
/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3038.MOV
/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3072.MOV
/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3094.MOV
/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_8567.MP4
/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_3095.MOV
/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_4157.MOV
/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_2987.MOV
/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_4155.MOV
/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_2988.MOV
/kaggle/input/include/Animals_1of2/Animals/4. Bird/MVI_4156.MOV
/kaggle/input/in

In [26]:
filename_pattern1 = os.path.join('/kaggle/input/include/Animals_1of2/Animals/4. Bird','MVI_2987.MOV')
tiled_astronaut_images = dask_image.imread.imread(filename_pattern)


UnknownFormatError: Could not detect your file type because it did not have an extension. Try specifying a loader class, e.g. Video(<generator object _iglob at 0x7f5ec4c5c1d0>)

In [43]:
tiled_astronaut_images

,Array,Chunk
Bytes,450.88 MiB,5.93 MiB
Shape,"(76, 1080, 1920, 3)","(1, 1080, 1920, 3)"
Count,228 Tasks,76 Chunks
Type,uint8,numpy.ndarray


In [20]:
filenames = glob.glob('/kaggle/input/include-50/Animals/4. Bird/*.MOV')


In [23]:
import imageio

In [25]:
lazy_arrays = [dask.delayed(imageio.imread)(fn) for fn in filenames]
#lazy_arrays = [da.from_delayed(x, shape=sample.shape, dtype=sample.dtype)
               #for x in lazy_arrays]

In [21]:
filenames

['/kaggle/input/include-50/Animals/4. Bird/MVI_3040.MOV',
 '/kaggle/input/include-50/Animals/4. Bird/MVI_3070.MOV',
 '/kaggle/input/include-50/Animals/4. Bird/MVI_3011.MOV',
 '/kaggle/input/include-50/Animals/4. Bird/MVI_2989.MOV',
 '/kaggle/input/include-50/Animals/4. Bird/MVI_3038.MOV',
 '/kaggle/input/include-50/Animals/4. Bird/MVI_3072.MOV',
 '/kaggle/input/include-50/Animals/4. Bird/MVI_3094.MOV',
 '/kaggle/input/include-50/Animals/4. Bird/MVI_3095.MOV',
 '/kaggle/input/include-50/Animals/4. Bird/MVI_4157.MOV',
 '/kaggle/input/include-50/Animals/4. Bird/MVI_2987.MOV',
 '/kaggle/input/include-50/Animals/4. Bird/MVI_4155.MOV',
 '/kaggle/input/include-50/Animals/4. Bird/MVI_2988.MOV',
 '/kaggle/input/include-50/Animals/4. Bird/MVI_4156.MOV',
 '/kaggle/input/include-50/Animals/4. Bird/MVI_3012.MOV',
 '/kaggle/input/include-50/Animals/4. Bird/MVI_3096.MOV',
 '/kaggle/input/include-50/Animals/4. Bird/MVI_3013.MOV',
 '/kaggle/input/include-50/Animals/4. Bird/MVI_3071.MOV',
 '/kaggle/inpu